# Creating the web scraper

In [1]:
#!pip install bs4

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import sklearn.metrics as sm


import re

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import csv
import requests
import json
from pandas.io.json import json_normalize
from urllib.request import urlopen
from bs4 import BeautifulSoup


from neo4j import GraphDatabase 
from py2neo import Graph,Node,Relationship

# Getting the dataset

In [2]:
data = pd.read_csv('../data/music-subset.csv')

In [3]:
refined_data = data.loc[(data["rank"] == 1)]

In [5]:
new_data = refined_data.reset_index(drop = True)

In [6]:
refined_data.shape

(6, 22)

In [71]:
index_names = new_data[new_data['trend'] == 'SAME_POSITION' ].index

In [72]:
new_data.drop(index_names, inplace=True)

In [73]:
new_data.shape

(3, 22)

In [74]:
new_data

,index,title,rank,date,artist,url,region,chart,trend,streams,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
1,201,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0,...,7.0,-7.028,1.0,0.2820,0.1650,0.000003,0.134,0.535,186.054,230453.0
2,402,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0,...,10.0,-3.194,1.0,0.0458,0.1860,0.000405,0.121,0.157,97.019,161200.0
3,603,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0,...,6.0,-4.582,0.0,0.0845,0.0154,0.000000,0.235,0.831,90.018,190813.0


In [77]:
contain_values = data[data.artist.str.contains('The Weeknd')]
print (contain_values)

     index                 title  rank        date  \
28      28               Starboy    29  2017-01-01   
75      75      I Feel It Coming    76  2017-01-01   
201    201               Starboy     1  2017-01-01   
207    207      I Feel It Coming     7  2017-01-01   
266    266         Party Monster    66  2017-01-01   
307    307               Rockin’   107  2017-01-01   
331    331    Can't Feel My Face   130  2017-01-01   
333    333             Sidewalks   132  2017-01-01   
368    368  Starboy - Kygo Remix   167  2017-01-01   
388    388        Six Feet Under   187  2017-01-01   
394    394             The Hills   193  2017-01-01   
408    408               Starboy     7  2017-01-01   
449    449      I Feel It Coming    47  2017-01-01   
560    560         Party Monster   158  2017-01-01   
605    605               Starboy     3  2017-01-01   
621    621      I Feel It Coming    19  2017-01-01   
725    725         Party Monster   123  2017-01-01   
819    819               Sta

In [76]:
#200403  I Feel It Coming
#209094   Save Your Tears
# 1 Starboy

## Reducing the amount of data

In [13]:
new_data.head()

,index,title,rank,date,artist,url,region,chart,trend,streams,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
1,201,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0,...,7.0,-7.028,1.0,0.2820,0.1650,0.000003,0.134,0.535,186.054,230453.0
2,402,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0,...,10.0,-3.194,1.0,0.0458,0.1860,0.000405,0.121,0.157,97.019,161200.0
3,603,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0,...,6.0,-4.582,0.0,0.0845,0.0154,0.000000,0.235,0.831,90.018,190813.0


In [14]:
#new_data = new_data.drop(new_data[new_data.index > 20].index)                          # Using logical condition
   

In [15]:
new_data.shape

(3, 22)

In [16]:
new_data.iloc[140087:140100,:]

,index,title,rank,date,artist,url,region,chart,trend,streams,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms


In [17]:
new_data

,index,title,rank,date,artist,url,region,chart,trend,streams,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
1,201,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0,...,7.0,-7.028,1.0,0.2820,0.1650,0.000003,0.134,0.535,186.054,230453.0
2,402,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0,...,10.0,-3.194,1.0,0.0458,0.1860,0.000405,0.121,0.157,97.019,161200.0
3,603,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0,...,6.0,-4.582,0.0,0.0845,0.0154,0.000000,0.235,0.831,90.018,190813.0


# Import dataset

In [7]:
data = pd.read_csv('../data/music-subset.csv', index_col=0)

In [9]:
data.head()

,title,rank,date,artist,url,region,chart,trend,streams,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
index,,,,,,,,,,,,,,,,,,,,,
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0,0.852,...,8.0,-2.921,0.0,0.0776,0.18700,0.000030,0.1590,0.907,102.034,195840.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0,0.663,...,11.0,-4.070,0.0,0.2260,0.00431,0.000017,0.1010,0.533,99.935,259196.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0,0.761,...,4.0,-3.073,0.0,0.0502,0.40000,0.000000,0.1760,0.710,93.974,222560.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0,0.508,...,0.0,-4.361,1.0,0.3260,0.55100,0.000003,0.1260,0.555,180.044,205600.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0,0.899,...,6.0,-4.228,0.0,0.2920,0.07600,0.000000,0.0631,0.873,88.007,234320.0


# Extracting artist and track name

In [2]:
data = pd.read_csv('../data/cleanedDatasetWithFeatures.csv')

In [3]:
data.drop('Unnamed: 0', inplace=True, axis=1)
data.drop('Unnamed: 0.1', inplace=True, axis=1)

In [4]:
data.head()

,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0776,0.187,0.000030,0.1590,0.9070,102.034,195840.0
1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,0.1700,0.209,0.000000,0.1120,0.8460,177.833,228200.0
2,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0470,0.426,0.000001,0.1440,0.4640,81.922,274000.0
3,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.2140,0.542,0.000013,0.0595,0.4420,96.507,417920.0
4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,0.0320,0.423,0.000002,0.4350,0.0979,129.863,221538.0


In [18]:
data.shape

(104561, 18)

In [85]:
track_nr = 56

In [5]:
import re

def get_track_name(nr):
    

    track_string = str(data.title[nr])
    
    x = track_string.find(" (")
    y = track_string.find(" -")


    
    if(int(x) < 0):
        track_string = track_string
    if(int(x) > 0):
        track_string = track_string[:x]
    if (int(y) > 0):
        track_string = track_string[:y]
    
    return track_string



def get_track_artist(nr):
    
    artist_string = str(data.artist[nr])

    y = artist_string.find(", ")
    x = artist_string.find(" (")
    
    if(int(x) > 0):
        artist_string = artist_string.replace("' ", "")

    if(int(y) > 0):
        #artist_string = y
        artist_string = artist_string[:y]     
        
    return artist_string

In [87]:
get_track_name(track_nr)

'drivers license'

In [88]:
get_track_artist(track_nr)

'Olivia Rodrigo'

# Getting URL

In [6]:
import re

def get_url(track_nr):
    
    track = str(get_track_name(track_nr))
    artist = str(get_track_artist(track_nr))
    
    track = re.sub('[^a-zA-Z0-9 \n\.]', '', track)
    artist = re.sub('[^a-zA-Z0-9 \n\.]', '', artist)
    
    
    if (int(artist.find(" ")) > 0):
        artist = artist.replace(" ", "-")
        
    if (int(track.find(" ")) > 0):
        track = track.replace(" ", "-")
        
    
    
    url_string = str('https://genius.com/' + artist + '-' + track + '-lyrics')
    
    return str(url_string)
    

In [90]:
get_url(track_nr)

'https://genius.com/Olivia-Rodrigo-drivers-license-lyrics'

# Scraping credits

In [7]:
def extract_credits(URL):
    
    # "https://genius.com/Shakia-chantaje-lyrics" <- url to check for raised exception
    
    r = requests.get(URL)
    
    if (int(r.status_code) == 200):


        # Parsing the HTML
        soup = BeautifulSoup(r.content, 'html.parser')
        s = soup.find('div', class_='SongInfo__Columns-nekw6x-2 lgBflw')
        g = soup.findAll(class_='SongTags__Tag-xixwg3-2 fdHeQh')

        feature = []
        writer = []
        label = []
        producer = []
        genre = []

        for line in s:
            div = line.find('div').text
            an = line.find_all('a')

            if 'Featuring' in div: 
                extract = [x.text for x in an]
                for x in extract:
                    feature.append(x)

            if 'Written By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    writer.append(x)

            if 'Label' in div:
                extract = [x.text for x in an]
                for x in extract:
                    label.append(x)

            if 'Produced By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    producer.append(x)

        for line in g:
            genre.append(line.text)
                    
    else:
        raise Exception("Sorry, no numbers below zero") 

    collection = {"Feature":feature, "Writer":writer, "Label":label, "Producer":producer, "Genres": genre}        
        
        
    return collection

In [92]:
#extLis = extract_credits(get_url(218900))
#extLis = extract_credits(get_url(200403))
#extLis = extract_credits(get_url(98))
extLis = extract_credits(get_url(track_nr))

# Dict to csv

In [97]:
#credit_list = pd.DataFrame.from_dict(extLis, orient='index').T
credit_list = pd.DataFrame.from_dict(extLis, orient='index').T

In [98]:
credit_list

,Feature,Writer,Label,Producer,Genres
0,None,Dan Nigro,Geffen Records,Dan Nigro,None
1,None,Olivia Rodrigo,Interscope Records,None,None


### Path to neo4j import folder
Nicholas Stationær: C:\Users\Nmtur\.Neo4jDesktop\relate-data\dbmss\dbms-20b18527-223b-4ada-8e62-429ad982fc9b\import\collaborators.csv\
Martins bærbare: C:\Users\marti\.Neo4jDesktop\relate-data\dbmss\dbms-a7f58857-34e9-48ad-b898-3dc31e11d74c\import\collaborators.csv

In [8]:
df = pd.DataFrame.from_dict(credit_list) 
df.to_csv (r'C:\Users\Nmtur\.Neo4jDesktop\relate-data\dbmss\dbms-20b18527-223b-4ada-8e62-429ad982fc9b\import\collaborators.csv', index = False, header=True)

NameError: name 'credit_list' is not defined

In [100]:
df

,Feature,Writer,Label,Producer,Genres
0,None,Dan Nigro,Geffen Records,Dan Nigro,None
1,None,Olivia Rodrigo,Interscope Records,None,None


***

# Neo4j

In [9]:
# Create connection to database

class neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
                print("Failed to create the driver: ", e)
                
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed!", e)
        finally:
            if session is not None:
                session.close()
        return response
    
    def close(self):
        if self.__driver is not None:
            self.__driver.close()

In [10]:
# Connecting to database
conn = neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="test123")


In [62]:
conn.query('''MATCH (n) RETURN n;''')

[<Record n=<Node id=0 labels=frozenset({'Person'}) properties={'name': 'Ricky Martin'}>>,
 <Record n=<Node id=1 labels=frozenset({'Person'}) properties={'name': 'Piso 21'}>>,
 <Record n=<Node id=2 labels=frozenset({'Person'}) properties={'name': 'Pablo Mejía'}>>,
 <Record n=<Node id=3 labels=frozenset({'Song'}) properties={'name': 'Gyal You A Party Animal'}>>,
 <Record n=<Node id=4 labels=frozenset({'Person'}) properties={'name': 'Juan David Huertas'}>>,
 <Record n=<Node id=5 labels=frozenset({'Person'}) properties={'name': 'Thalia'}>>,
 <Record n=<Node id=6 labels=frozenset({'Person'}) properties={'name': 'Ronny Svendsen'}>>,
 <Record n=<Node id=7 labels=frozenset({'Person'}) properties={'name': 'Marala'}>>,
 <Record n=<Node id=8 labels=frozenset({'Person'}) properties={'name': 'Pablo Uribe Trujillo'}>>,
 <Record n=<Node id=9 labels=frozenset({'Person'}) properties={'name': 'Sergio George'}>>,
 <Record n=<Node id=10 labels=frozenset({'Label'}) properties={'name': 'Sony Music Latin'}>>

In [64]:
ranking = data.iloc[track_nr][1]
artist = get_track_artist(track_nr)
track = get_track_name(track_nr)
track

'Perfect'

In [65]:
artist

'Ed Sheeran'

In [66]:
ranking

1

### Query to insert into neo4j

In [36]:
def run_querys(track_nr):
    
    artist = get_track_artist(track_nr)
    track = get_track_name(track_nr)
    ranking = str(data.iloc[track_nr][1])
    
    conn.query('''CALL apoc.load.csv('collaborators.csv')
    YIELD lineNo, map, list
    
    WHERE map.Feature IS NOT NULL
    MERGE (f:Person {name:map.Feature})
    MERGE (a:Person {name:"'''+ artist + '''"})
    MERGE (w:Person {name:map.Writer})
    MERGE (p:Person {name:map.Producer})
    MERGE (l:Label {name:map.Label})
    MERGE (s:Song {name:"'''+ track +'''", rank: "'''+ ranking + '''"})
    
    
    MERGE (f)-[:COLLABORATED]-(a)
    MERGE (f)-[:FEATURED]->(s)
    MERGE (a)-[:SANG]->(s)
    MERGE (w)-[:WROTE]->(s)
    MERGE (l)-[:RELEASED]->(s)
    MERGE (p)-[:PRODUCED]->(s);
    ''')
    
    conn.query('''MATCH (f:Person) WHERE (f.name = "") detach delete f;''')
    
    conn.query('''MATCH (l:Label) WHERE (l.name = "") detach delete l;''')
    
    conn.close()

In [79]:
run_querys2(track_nr)

In [18]:
errors = 0
for x in range(10) :
    try:
        extLis = extract_credits(get_url(x))
 #   print(str(get_track_artist(x)) + ' - ' + str(get_track_name(x)))
        print(extLis)
    except:
        #append error or insert specific values?
        errors = errors+1
        pass
    

{'Feature': ['Maluma'], 'Writer': ['Kevin ADG', 'Chan El Genio', 'Kenai', 'Maluma', 'Shakira'], 'Label': [], 'Producer': ['The Rudeboyz', 'Shakira', 'Maluma'], 'Genres': []}
{'Feature': ['Daddy Yankee'], 'Writer': ['Daddy Yankee', 'Erika Ender', 'Luis Fonsi'], 'Label': [], 'Producer': ['Andrés Torres', 'Mauricio Rengifo'], 'Genres': ['Pop']}
{'Feature': [], 'Writer': ['Zack Tabudlo'], 'Label': [], 'Producer': ['Zack Tabudlo'], 'Genres': []}
{'Feature': [], 'Writer': ['Ariana Grande', 'Richard Rodgers', 'TBHits', 'Njomza', 'Michael “Mikey” Foster', 'Kaydence', 'Tayla Parx', 'Scootie', 'Oscar Hammerstein II', 'Victoria Monét'], 'Label': ['Universal Music Group', 'Republic Records'], 'Producer': ['Scootie', 'Michael “Mikey” Foster', 'TBHits'], 'Genres': ['Rap', 'R&B', 'Pop']}
{'Feature': [], 'Writer': ['Steve Mac', 'Tameka “Tiny” Harris', 'Kandi', 'Kevin Briggs', 'Ed Sheeran', 'Johnny McDaid'], 'Label': ['Asylum Records', 'Atlantic Records'], 'Producer': ['Steve Mac'], 'Genres': []}
{'Fea

In [66]:
df

,Feature,Writer,Label,Producer,Genres
0,None,Zack Tabudlo,None,Zack Tabudlo,None


### Getting the Shit

In [28]:
def running(track_nr):
    # For scraping
    extLis = extract_credits(get_url(track_nr))
    # Get credit
    credit_list = pd.DataFrame.from_dict(extLis, orient='index').T
    df = pd.DataFrame.from_dict(credit_list) 
    df.to_csv (r'C:\Users\Nmtur\.Neo4jDesktop\relate-data\dbmss\dbms-20b18527-223b-4ada-8e62-429ad982fc9b\import\collaborators.csv', index = False, header=True)
    # Running neo4j query
    run_querys(track_nr)


In [44]:
import time
tic = time.perf_counter()

errors = 0
count = 0
for i in range(1000):
    try:
        running(i)
        print(str(int(((index+1)/data.shape[0])*100)) + '% done - ' + str(index+1) + ' of ' + str(data.shape[0]) + ' songs iterated - ' + errors + ' encountered.', end='\r')
    except:
        #append error or insert specific values?
        errors = errors+1
        pass
    count = count+1
    print(count, end='\r')
        
toc = time.perf_counter()
print(f"Added new rows from external links in {toc - tic:0.4f} seconds")

Added new rows from external links in 2224.4638 seconds
